### Agents
Agents will provide a way for us to add additional sources of information that will build on our RAG approach.  We may need to execute code or search different information sources to build a complex context to answer user queries.

### Tools

In [ ]:
%pip install langchain_community -q
%pip install tavily-python -q
%pip install python-dotenv -q
%pip install langchainhub -q
%pip install wikibase-rest-api-client mediawikiapi -q

In [ ]:
import os
import boto3

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import AIMessage, HumanMessage
from langchain import hub
from dotenv import load_dotenv

load_dotenv()
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

### Agent Setup
We need three components to make up our agent
1) Tools we plan to use to execute respond to input
2) LLM we plan to use as our logic executor
3) Agent type we plan to use for this use case


#### Homework
Find a useful tool other than Wikidata or Tavily to explore what it can do

In [2]:
# Your tool here
# 1 Start with a simple web search tool:
#tools = -> FIND A GOOD TOOL

In [ ]:
# 2 LLM, Let's use Claude's Haiku
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Grab our LLM of choice
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 0.9,
    "stop_sequences": ["\n\nHuman"],
}
llm = BedrockChat(
    client=aws_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Get the prompt to use with our agent
# The hub provides sample templates for each agent type
prompt = hub.pull("hwchase17/structured-chat-agent")

# print("Tools:", tools)
# print("Prompt", prompt)



Using your new tool see if you can get the expected output

In [ ]:
# 3 Construct the create_structured_chat_agent agent
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
agent = create_structured_chat_agent(llm, tools, prompt)

# run the agent
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

agent_executor.invoke({"input": "What happened with the economy today?"})